In [1]:
from transformers import T5ForConditionalGeneration, T5Tokenizer


libgomp: Invalid value for environment variable OMP_NUM_THREADS


In [4]:
model_name = 'google/madlad400-3b-mt'
model = T5ForConditionalGeneration.from_pretrained(model_name, device_map="auto")
tokenizer = T5Tokenizer.from_pretrained(model_name)

Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at google/madlad400-3b-mt and are newly initialized: ['shared.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [6]:
text = """1) Кожна людина має право на освіту. Освіта повинна бути безплатною, хоча б початкова і загальна. Початкова освіта повинна бути обов'язковою. Технічна і професійна освіта повинна бути загальнодоступною, а вища освіта повинна бути однаково доступною для всіх на основі здібностей кожного.
2) Освіта повинна бути спрямована на повний розвиток людської особи і збільшення поваги до прав людини і основних свобод. Освіта повинна сприяти взаєморозумінню, терпимості і дружбі між усіма народами, расовими, або релігійними групами і повинна сприяти діяльності Організації Об'єднаних Націй по підтриманню миру.
3) Батьки мають право пріоритету у виборі виду освіти для своїх малолітніх дітей."""

input_ids = tokenizer(text, return_tensors="pt").input_ids.to(model.device)
outputs = model.generate(input_ids=input_ids)

tokenizer.decode(outputs[0], skip_special_tokens=True)

'2) Освіта повинна бути безкоштовною, хоча б початкова'

In [7]:
input_ids

tensor([[   805,     38,    805,    808,    830,  86049,   1215,  98325,   1215,
          89278,  25805,   1038, 181142,   1047,    807,  30812,  50956,    914,
         160715,    914,  57752,   6116, 121746,  54695,    806, 150796,   2251,
           1569,  28298,  13887,   1715, 213851,    914,    807, 125516,   1674,
          13887, 181142,    914, 160715,    914,  57752,    805, 176043,    814,
         227976,  15502,    807, 191919, 209908,   1715, 188491,  16909,   1215,
         181142,    914, 160715,    914,  57752, 213851,  34204,  45254,  54695,
            806,   1158,   1179, 115554, 181142,    914, 160715,    914,  57752,
          75813,  90574,  76912,  54695,   4090,  76747,   1190,   1038,  45312,
           1108,   1598,   8655,   1553,    953, 191953,  80721,   9052,    807,
              3,    810,    830,  30812,  50956,    914, 160715,    914,  57752,
         112710,  13083, 205871,   1038,  24869,   5058, 197093,  44637, 134696,
          60658, 174837,   1

In [13]:
import torch

In [11]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

In [9]:
model = model.to('cuda')

In [36]:
article_uk = 

In [42]:
tokenizer.src_lang = "uk_UA"
encoded_hi = tokenizer(article_uk, return_tensors="pt").to('cuda')

with torch.no_grad():
    generated_tokens = model.generate(
        **encoded_hi,
        forced_bos_token_id=tokenizer.lang_code_to_id["fr_XX"]
    )

In [43]:
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

["1) Every personne a droit à l'éducation. L'éducation doit être gratuite, à la fois primaire et générale. L'éducation doit être obligatoire. L'éducation technique et professionnelle doit être accessible à tous, à tous les niveaux, sur la base des capacités de chacun. 2) L'éducation doit être axée sur le plein développement de la personne humaine et l'accroissement des droits de la personne et des libertés fondamentales. L'éducation doit favoriser l'interdiction, la tolérance et l'amitié entre tous les peuples, qu'il s'agisse de races ou de religions, et doit favoriser l'action de l'Organisation des Nations Unies pour le maintien de la paix. 3) Les parents ont le droit de privilégier l'éducation pour leurs enfants."]

---

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-uk-ru")

model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-uk-ru").to('cuda')


libgomp: Invalid value for environment variable OMP_NUM_THREADS
/home/user/conda/lib/python3.7/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [2]:
model.config.max_position_embeddings, sum(p.numel() for p in model.parameters())

(512, 74244608)

In [3]:
!nvidia-smi

Sun Dec 24 13:45:44 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  | 00000000:C0:00.0 Off |                    0 |
| N/A   26C    P0              76W / 400W |    855MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [4]:
def translate(text):
    input_ids = tokenizer(text, return_tensors="pt").input_ids.to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids)
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def translate_batch(texts):
    input_ids = tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(**input_ids)
    
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [5]:
import numpy as np
import pandas as pd

In [6]:
df = pd.read_csv("ru_uk.tsv", sep='\t')

In [7]:
df.head()

,index,family_id,src_lang,src_abs,tgt_lang,tgt_abs
0,3431,52281850,ru,Способ выращивания капусты краснокочанной с пр...,uk,Спосіб вирощування капусти червоноголової з за...
1,3041,50556686,ru,"Способ утилизации тепла горячих газов, отходящ...",uk,"Спосіб утилізації теплоти гарячих газів, що ві..."
2,2364,40259183,ru,Описаны устройства и способы для управления вы...,uk,Описані пристрої та способи для керування вияв...
3,3050,50557348,ru,Белково-технологическая композиция содержит жи...,uk,Білково-технологічна композиція містить тварин...
4,2142,38063660,ru,В заявке описаны сульфонилэтил- и тиоэтилфосфо...,uk,У заявці описані сульфонілетил- і тіоетилфосфо...


In [8]:
df.dropna(inplace=True)

In [9]:
df['tgt_abs']

0        Спосіб вирощування капусти червоноголової з за...
1        Спосіб утилізації теплоти гарячих газів, що ві...
2        Описані пристрої та способи для керування вияв...
3        Білково-технологічна композиція містить тварин...
4        У заявці описані сульфонілетил- і тіоетилфосфо...
                               ...                        
85958    Пробовідбірник для рідин містить накопичувальн...
85959    Комплексний резонансний глушник шуму, який міс...
85960    Спосіб передопераційної підготовки хворих до р...
85961    Спосіб підвищення інтенсивності росту і розвит...
85962    Одноразова ємкість містить корпус у вигляді ци...
Name: tgt_abs, Length: 85963, dtype: object

In [10]:
import sys
from tqdm import tqdm

In [11]:
# 90mb -> 100Gb
sys.getsizeof(df['tgt_abs']) / 1024 ** 2

91.28665161132812

In [12]:
translate(df['tgt_abs'].iloc[0])

/home/user/conda/lib/python3.7/site-packages/transformers/generation/utils.py:1292: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,
/home/user/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:585: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.object,
/home/user/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:637: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify 

'Способ выращивания капусты красноголовой с применением эм-препарата включает обработка данного лекарства почвы к посеву с нормой 20 л/га. Наматывают семена с нормой 1 л/т и делают внекорректные поделения во время вегетации с нормой 2 л/га в три периода.'

In [13]:
translate(df['tgt_abs'].iloc[-1])

/home/user/conda/lib/python3.7/site-packages/transformers/generation/utils.py:1292: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


'Одноразовая ёмкость содержит корпус в виде цилиндра, разделенного на камеры, крышку и перегородку.'

In [14]:
translate_batch(
    [df['tgt_abs'].iloc[0],
     df['tgt_abs'].iloc[-1]]
)

['Способ выращивания капусты красноголовой с применением эм-препарата включает обработка данного лекарства почвы к посеву с нормой 20 л/га. Наматывают семена с нормой 1 л/т и делают внекорректные поделения во время вегетации с нормой 2 л/га в три периода.',
 'Одноразовая ёмкость содержит корпус в виде цилиндра, разделенного на камеры, крышку и перегородку.']

In [15]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [16]:
bs = 16

for chunk in tqdm(
    batch(df['tgt_abs'].tolist(), bs),
    total=len(df['tgt_abs']) // bs
):
    translate_batch(chunk)
    torch.cuda.empty_cache()

  0%|          | 10/5372 [00:34<5:11:33,  3.49s/it]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:7                                                                                    │
│                                                                                                  │
│   4 │   batch(df['tgt_abs'].tolist(), bs),                                                       │
│   5 │   total=len(df['tgt_abs']) // bs                                                           │
│   6 ):                                                                                           │
│ ❱ 7 │   translate_batch(chunk)                                                                   │
│   8 │   torch.cuda.empty_cache()                                                                 │
│   9                                                                                              │
│                                                                                                  │
│ in translate_batch:13                                                                            │
│                                                                                                  │
│   10 │   input_ids = tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(    │
│   11 │                                                                                           │
│   12 │   with torch.no_grad():                                                                   │
│ ❱ 13 │   │   outputs = model.generate(**input_ids)                                               │
│   14 │                                                                                           │
│   15 │   return tokenizer.batch_decode(outputs, skip_special_tokens=True)                        │
│   16                                                                                             │
│                                                                                                  │
│ /home/user/conda/lib/python3.7/site-packages/torch/autograd/grad_mode.py:27 in decorate_context  │
│                                                                                                  │
│    24 │   │   @functools.wraps(func)                                                             │
│    25 │   │   def decorate_context(*args, **kwargs):                                             │
│    26 │   │   │   with self.clone():                                                             │
│ ❱  27 │   │   │   │   return func(*args, **kwargs)                                               │
│    28 │   │   return cast(F, decorate_context)                                                   │
│    29 │                                                                                          │
│    30 │   def _wrap_generator(self, func):                                                       │
│                                                                                                  │
│ /home/user/conda/lib/python3.7/site-packages/transformers/generation/utils.py:1500 in generate   │
│                                                                                                  │
│   1497 │   │   │   │   output_scores=generation_config.output_scores,                            │
│   1498 │   │   │   │   return_dict_in_generate=generation_config.return_dict_in_generate,        │
│   1499 │   │   │   │   synced_gpus=synced_gpus,                                                  │
│ ❱ 1500 │   │   │   │   **model_kwargs,                                                           │
│   1501 │   │   │   )                                                                             │
│   1502 │   │                                                                                     │
│   1503 │   │   elif is_beam_sample_gen_mode:                                                     │
│                                                            

In [22]:
# во сколько раз данных больше
100 * 1024 / 91

1125.2747252747254

In [23]:
# количество дней на 1 карте
5 * 1125 / 24

234.375

In [24]:
# количество дней на 16 картах
234 / 16

14.625